# MWE polygene

In [3]:
import sys

In [4]:
script_template = r'''module load gcc/10.2.0
module load R

IFS="
"

cmd="Rscript {$FILENAME}"
bsub -W 2440 -R "rusage[mem=100]" -e scripts/{$NAME}.err -o scripts/{$NAME}.out -n 1 "$cmd"
'''

In [7]:
r_template_poly = r'''library(data.table)
library(fgsea)
gxt2 = read.table(paste0("gene_scores/PolyGene_PPI_PoPS_justMAGMA_Mean.txt"))

autoimmune = c("PASS_Celiac", "PASS_IBD_deLange2017", "PASS_CD_deLange2017", "PASS_UC_deLange2017",
               "PASS_Rheumatoid_Arthritis", "PASS_Lupus", "UKB_460K.disease_ALLERGY_ECZEMA_DIAGNOSED",
               "PASS_Type_1_Diabetes", "PASS_Multiple_sclerosis", "PASS_Alzheimers_Jansen2019",
               "PASS_Primary_biliary_cirrhosis",  "UKB_460K.disease_AID_ALL")        # autoimmune diseases

brain = c("UKB_460K.other_MORNINGPERSON", "PASS_Neuroticism",
               "PASS_Schizophrenia", "PASS_SCZvsBD_Ruderfer2018",
               "PASS_SleepDuration_Dashti2019", "PASS_Years_of_Education1",
               "PASS_Autism", "PASS_BipolarDisorder_Ruderfer2018", "PASS_BMI1", "PASS_Depression_Nagel2018",
               "PASS_Insomnia_Jansen2019", "PASS_Intelligence_SavageJansen2018", "PASS_MDD_Wray2018")      # brain diseases

blood = c("UKB_460K.blood_EOSINOPHIL_COUNT",
               "UKB_460K.blood_LYMPHOCYTE_COUNT",
               "UKB_460K.blood_MEAN_CORPUSCULAR_HEMOGLOBIN",
               "UKB_460K.blood_PLATELET_COUNT",
               "UKB_460K.blood_MONOCYTE_COUNT",
               "UKB_460K.blood_RBC_DISTRIB_WIDTH",
               "UKB_460K.blood_RED_COUNT",
               "UKB_460K.blood_WHITE_COUNT") # blood cell traits

allpoly = colnames(gxt2)

traitnames = {$TRAITS}

## Loading in the gene program files ####

ff = list.files("clusters/{$DS}/{$TP}")
ll=list()
for(numf in 1:length(ff)){
  genes_df = read.table(paste0("clusters/{$DS}/{$TP}/", "/",
                               ff[numf]))
  ll[[numf]] = genes_df[,1]
}
names(ll) = as.character(sapply(ff, function(x) return(strsplit(x, "_top")[[1]][1])))

numvec = sapply(ll, function(y) return(length(y)))

GSEA_jointENR = c()
GSEA_jointpENR = c()
GSEA_leadEdge = c()
ENR_mat = c()
NENR_mat = c()
pENR_mat = c()
for(traitname in traitnames){

  scores2 = gxt2[, traitname]
  names(scores2) = rownames(gxt2)

  #fgsea_res <- fgsea(ll, scores2, scoreType = "pos",
  #                   minSize=15,
  #                   nproc=4)
  #NESvec = as.numeric(as.data.frame(fgsea_res)[,6])
  #pval_vec = as.numeric(as.data.frame(fgsea_res)[,2])
  #leading_edge = unlist(lapply(fgsea_res$leadingEdge, function(x) return(paste0(x[1:5], collapse = ";"))))

  enr = sapply(ll, function(x){
    temp = intersect(x, names(scores2))
    yy = mean(scores2[temp]^2)/mean(scores2^2)
    return(yy)
  })

  total_genes = unique(unlist(ll))
  boot_enrvec = c()
  for(mm in 1:5000){
    numsamp = as.numeric(sample(numvec, 1))
    temp2 = sample(total_genes, numsamp, replace = T)
    temp2 = intersect(temp2, names(scores2))
    yy = mean(scores2[temp2]^2)/mean(scores2^2)
    boot_enrvec = c(boot_enrvec, yy)
  }

  penr = c()
  for(mm in 1:length(enr)){
    penr[mm] = pnorm(enr[mm], median(boot_enrvec), sd(boot_enrvec), lower.tail = F)
  }
  enr2 = enr/median(boot_enrvec)

  ENR_mat = cbind(ENR_mat, enr)
  NENR_mat = cbind(NENR_mat, enr2)
  pENR_mat = cbind(pENR_mat, penr)
  #GSEA_jointENR = cbind(GSEA_jointENR, NESvec)
  #GSEA_jointpENR = cbind(GSEA_jointpENR, pval_vec)
  #GSEA_leadEdge = cbind(GSEA_leadEdge, leading_edge)
  cat("We are at trait:", traitname, "\n")
}

                               
rownames(ENR_mat) = names(ll)
rownames(NENR_mat) = names(ll)
rownames(pENR_mat) = names(ll)
#rownames(GSEA_jointENR) = names(ll)
#rownames(GSEA_jointpENR) = names(ll)
#rownames(GSEA_leadEdge) = names(ll)

colnames(ENR_mat) = traitnames
colnames(NENR_mat) = traitnames
colnames(pENR_mat) = traitnames
#colnames(GSEA_jointENR) = traitnames
#colnames(GSEA_jointpENR) = traitnames
#colnames(GSEA_leadEdge) = traitnames

outll = list("ENR" = ENR_mat,
             "NENR" = NENR_mat,
             "pENR" = pENR_mat)#,
             #"GSEA_jointENR" = GSEA_jointENR,
             #"GSEA_jointpENR" = GSEA_jointpENR,
             #"GSEA_leadEdge" = GSEA_leadEdge)

save(outll, file = "polygene_pops/{$TP}_{$TRAITS}_poly.rda") #### CHANGE PATH
'''



r_template_pops = r'''library(data.table)
library(fgsea)
gxt2 = read.table("gene_scores/POPs_MAGMA_0kb.txt")

autoimmune = c("PASS_Celiac", "PASS_IBD_deLange2017", "PASS_CD_deLange2017", "PASS_UC_deLange2017",
               "PASS_Rheumatoid_Arthritis", "PASS_Lupus", "UKB_460K.disease_ALLERGY_ECZEMA_DIAGNOSED",
               "PASS_Type_1_Diabetes", "PASS_Multiple_sclerosis", "PASS_Alzheimers_Jansen2019",
               "PASS_Primary_biliary_cirrhosis",  "UKB_460K.disease_AID_ALL")        # autoimmune diseases

brain = c("UKB_460K.other_MORNINGPERSON", "PASS_Neuroticism",
               "PASS_Schizophrenia", "PASS_SCZvsBD_Ruderfer2018",
               "PASS_SleepDuration_Dashti2019", "PASS_Years_of_Education1",
               "PASS_Autism", "PASS_BipolarDisorder_Ruderfer2018", "PASS_BMI1", "PASS_Depression_Nagel2018",
               "PASS_Insomnia_Jansen2019", "PASS_Intelligence_SavageJansen2018", "PASS_MDD_Wray2018")      # brain diseases

blood = c("UKB_460K.blood_EOSINOPHIL_COUNT",
               "UKB_460K.blood_LYMPHOCYTE_COUNT",
               "UKB_460K.blood_MEAN_CORPUSCULAR_HEMOGLOBIN",
               "UKB_460K.blood_PLATELET_COUNT",
               "UKB_460K.blood_MONOCYTE_COUNT",
               "UKB_460K.blood_RBC_DISTRIB_WIDTH",
               "UKB_460K.blood_RED_COUNT",
               "UKB_460K.blood_WHITE_COUNT") # blood cell traits

allpoly = colnames(gxt2)

traitnames = {$TRAITS}

## Loading in the gene program files ####

ff = list.files("clusters/{$DS}/{$TP}")
ll=list()
for(numf in 1:length(ff)){
  genes_df = read.table(paste0("clusters/{$DS}/{$TP}/", "/",
                               ff[numf]))
  ll[[numf]] = genes_df[,1]
}
names(ll) = as.character(sapply(ff, function(x) return(strsplit(x, "_top")[[1]][1])))

numvec = sapply(ll, function(y) return(length(y)))

GSEA_jointENR = c()
GSEA_jointpENR = c()
GSEA_leadEdge = c()
ENR_mat = c()
NENR_mat = c()
pENR_mat = c()
for(traitname in traitnames){

  scores2 = gxt2[, traitname]
  names(scores2) = rownames(gxt2)

  #fgsea_res <- fgsea(ll, scores2, scoreType = "pos",
  #                   minSize=15,
  #                   nproc=4)
  #NESvec = as.numeric(as.data.frame(fgsea_res)[,6])
  #pval_vec = as.numeric(as.data.frame(fgsea_res)[,2])
  #leading_edge = unlist(lapply(fgsea_res$leadingEdge, function(x) return(paste0(x[1:5], collapse = ";"))))

  enr = sapply(ll, function(x){
    temp = intersect(x, names(scores2))
    yy = mean(scores2[temp]^2)/mean(scores2^2)
    return(yy)
  })

  total_genes = unique(unlist(ll))
  boot_enrvec = c()
  for(mm in 1:5000){
    numsamp = as.numeric(sample(numvec, 1))
    temp2 = sample(total_genes, numsamp, replace = T)
    temp2 = intersect(temp2, names(scores2))
    yy = mean(scores2[temp2]^2)/mean(scores2^2)
    boot_enrvec = c(boot_enrvec, yy)
  }

  penr = c()
  for(mm in 1:length(enr)){
    penr[mm] = pnorm(enr[mm], median(boot_enrvec), sd(boot_enrvec), lower.tail = F)
  }
  enr2 = enr/median(boot_enrvec)

  ENR_mat = cbind(ENR_mat, enr)
  NENR_mat = cbind(NENR_mat, enr2)
  pENR_mat = cbind(pENR_mat, penr)
  #GSEA_jointENR = cbind(GSEA_jointENR, NESvec)
  #GSEA_jointpENR = cbind(GSEA_jointpENR, pval_vec)
  #GSEA_leadEdge = cbind(GSEA_leadEdge, leading_edge)
  cat("We are at trait:", traitname, "\n")
}

                               
rownames(ENR_mat) = names(ll)
rownames(NENR_mat) = names(ll)
rownames(pENR_mat) = names(ll)
#rownames(GSEA_jointENR) = names(ll)
#rownames(GSEA_jointpENR) = names(ll)
#rownames(GSEA_leadEdge) = names(ll)

colnames(ENR_mat) = traitnames
colnames(NENR_mat) = traitnames
colnames(pENR_mat) = traitnames
#colnames(GSEA_jointENR) = traitnames
#colnames(GSEA_jointpENR) = traitnames
#colnames(GSEA_leadEdge) = traitnames

outll = list("ENR" = ENR_mat,
             "NENR" = NENR_mat,
             "pENR" = pENR_mat)#,
             #"GSEA_jointENR" = GSEA_jointENR,
             #"GSEA_jointpENR" = GSEA_jointpENR,
             #"GSEA_leadEdge" = GSEA_leadEdge)

save(outll, file = "polygene_pops/{$TP}_{$TRAITS}_pops.rda") #### CHANGE PATH
'''

In [6]:
def write_script(ds, tp, trait, ispoly=True): #ispoly determines which gene scores to use, the PoPS gene scores or PolyGene scores
    if ispoly:
        NAME = f"{ds}_{tp}_{trait}_poly"
    else:
        NAME = f"{ds}_{tp}_{trait}_pops"

    FILENAME = f"scripts/{NAME}.R"
    
    temp = script_template.replace("{$NAME}", NAME)
    temp = temp.replace("{$FILENAME}", FILENAME)
    with open(f"scripts/{NAME}.sh", "w") as f:
        f.write(temp)
    print(f"{NAME}.sh")
    
    if ispoly:
        r_template = r_template_poly
    else:
        r_template = r_template_pops
    temp = r_template.replace("{$TRAITS}", trait)
    temp = temp.replace("{$TP}", tp)
    temp = temp.replace("{$DS}", ds)
    with open(f"scripts/{NAME}.R", "w") as f:
        f.write(temp)
    print(f"{NAME}.R")
    print()

In [ ]:
write_script("mwe_data", "all_pmdfiltered_top1000", "allpoly")